In [1]:
import pandas as pd
import numpy as np

In [2]:
df_enga = pd.read_csv('takehome_user_engagement.csv')
df_enga[df_enga.user_id == 2]

,time_stamp,user_id,visited
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1
10,2014-02-13 03:45:04,2,1


In [3]:
df = pd.read_csv('takehome_users.csv',encoding = "ISO-8859-1")
df.head(10)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0


#### For the first data set ('takehome_user_engagement.csv'), I grouped all the users data and resample them into a 7-days period. And I searched for users if there is more that 3 visits in a single period. And it became labels for later model developing.

In [4]:
df_enga_new = pd.read_csv('takehome_user_engagement.csv')
df_enga_new['time_stamp'] = pd.to_datetime(df_enga_new.time_stamp)
df_enga_new = df_enga_new.set_index('time_stamp')
df_enga_new = df_enga_new.groupby(['user_id',pd.TimeGrouper('7D')]).count()
df_enga_new.head(20)

visited
user_id time_stamp                  
1       2014-04-17 08:20:06        1
2       2013-11-14 08:20:06        1
        2013-11-28 08:20:06        1
        2013-12-05 08:20:06        1
        2013-12-19 08:20:06        1
        2013-12-26 08:20:06        1
        2014-01-02 08:20:06        1
        2014-01-30 08:20:06        1
        2014-02-06 08:20:06        3
        2014-02-13 08:20:06        1
        2014-03-06 08:20:06        2
        2014-03-27 08:20:06        1
3       2013-03-14 08:20:06        1
4       2013-05-16 08:20:06        1
5       2013-01-17 08:20:06        1
6       2013-12-12 08:20:06        1
7       2012-12-20 08:20:06        1
10      2013-01-10 08:20:06        1
        2013-01-17 08:20:06        1
        2013-01-24 08:20:06        1

In [5]:
adopted = {}
for i in df_enga.user_id.unique():
    adopted[i] = df_enga_new.loc[i].visited.max() >= 3

#### For the second data set ('takehome_users.csv'), I firstly removed some unnecessary features, like email address, users' names. Then I appended the labels obtained before to each user.

In [6]:
df_user = pd.read_csv('takehome_users.csv',encoding = "ISO-8859-1")
df_user = df_user.set_index('object_id')
df_user['adopted'] = pd.Series(adopted, index = df_user.index)
df_user = df_user[df_user.adopted.notnull()]
drop_list = ['creation_time','name','email']
df_user = df_user.drop(drop_list,axis = 1)
df_user['invited_by_user_id'] = df_user['invited_by_user_id'].notnull()
df_user = df_user.reset_index()
df_user.head(10)

,object_id,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
0,1,GUEST_INVITE,1.398139e+09,1,0,11,True,False
1,2,ORG_INVITE,1.396238e+09,0,0,1,True,True
2,3,ORG_INVITE,1.363735e+09,0,0,94,True,False
3,4,GUEST_INVITE,1.369210e+09,0,0,1,True,False
4,5,GUEST_INVITE,1.358850e+09,0,0,193,True,False
5,6,GUEST_INVITE,1.387424e+09,0,0,197,True,False
6,7,SIGNUP,1.356010e+09,0,1,37,False,False
7,10,ORG_INVITE,1.401833e+09,1,1,318,True,True
8,11,SIGNUP,1.388117e+09,0,0,69,False,False
9,13,ORG_INVITE,1.396196e+09,0,0,254,True,False


### Building the label matrix

In [7]:
y = np.where(df_user['adopted'],1,0)
y

array([0, 1, 0, ..., 0, 0, 0])

#### Used One-Hot encoding method to build feature map for 'creation_source' feature, which I think it would be a critical feature.

In [9]:
s = df_user.creation_source
one_hot = pd.get_dummies(s)
df_user = pd.concat([df_user, one_hot], axis=1, join_axes=[df_user.index])
drop_list = ['object_id','creation_source','adopted','org_id']
df_user = df_user.drop(drop_list,axis = 1)
df_user.head(10)

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1.398139e+09,1,0,True,1,0,0,0,0
1,1.396238e+09,0,0,True,0,1,0,0,0
2,1.363735e+09,0,0,True,0,1,0,0,0
3,1.369210e+09,0,0,True,1,0,0,0,0
4,1.358850e+09,0,0,True,1,0,0,0,0
5,1.387424e+09,0,0,True,1,0,0,0,0
6,1.356010e+09,0,1,False,0,0,0,1,0
7,1.401833e+09,1,1,True,0,1,0,0,0
8,1.388117e+09,0,0,False,0,0,0,1,0
9,1.396196e+09,0,0,True,0,1,0,0,0


### Model Building and Evaluation

In [10]:
X = df_user.as_matrix().astype(np.float)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

print('Feature space holds {} observations and {} features'.format(X.shape[0],X.shape[1]))
print("Unique target labels:{}".format(np.unique(y)))

Feature space holds 8823 observations and 9 features
Unique target labels:[0 1]


In [11]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    clf = clf_class(**kwargs)
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

C:\Users\Chiroy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression

def accuracy(y_true,y_pred):
    return np.mean(y_true == y_pred)

LR_CV_result = run_cv(X,y,LogisticRegression)
LR_CV_L1_result = run_cv(X,y,LogisticRegression,penalty='l1')
RF_CV_result = run_cv(X,y,RandomForestClassifier)
KNN_CV_result = run_cv(X,y,KNeighborsClassifier)

print ('Accuracy')
print ("Logistic Regression L1: " + str(accuracy(y, LR_CV_L1_result)*100) + '%')
print ("Logistic Regression L2: " + str(accuracy(y, LR_CV_result)*100) + '%')
print ("Random forest: " + str(accuracy(y, RF_CV_result)*100) + '%')
print ("K-nearest-neighbors: " + str(accuracy(y, KNN_CV_result)*100) + '%')

Accuracy
Logistic Regression L1: 84.9710982659%
Logistic Regression L2: 84.6537458914%
Random forest: 87.5665873286%
K-nearest-neighbors: 89.8787260569%


In [13]:
forest = RandomForestClassifier()
forest.fit(X, y)

importances = forest.feature_importances_

# Print the feature ranking
print("Feature importance ranking by Random Forest Model:")
for k,v in sorted(zip(map(lambda x: round(x, 4), importances), df_user.columns), reverse=True):
    print(v + ": " + str(k))

Feature importance ranking by Random Forest Model:
last_session_creation_time: 0.9825
enabled_for_marketing_drip: 0.0055
opted_in_to_mailing_list: 0.0051
GUEST_INVITE: 0.0019
ORG_INVITE: 0.0014
invited_by_user_id: 0.001
PERSONAL_PROJECTS: 0.001
SIGNUP_GOOGLE_AUTH: 0.0008
SIGNUP: 0.0007


#### The model I firstly built gave me the result that the timestamp of last login would be the most important feature to predict. While in my opinion, this is not reasonable, because they are relatively independent events and the last login won't affect the future user adoption too much.

In [14]:
df_user_new = df_user.drop(['last_session_creation_time'],axis = 1)
X1 = df_user_new.as_matrix().astype(np.float)
X1 = scaler.fit_transform(X1)

LR_CV_result_1 = run_cv(X1,y,LogisticRegression)
LR_CV_L1_result_1 = run_cv(X1,y,LogisticRegression,penalty='l1')
RF_CV_result_1 = run_cv(X1,y,RandomForestClassifier)
KNN_CV_result_1 = run_cv(X1,y,KNeighborsClassifier)

print ('Accuracy')
print ("Logistic Regression L1: " + str(accuracy(y, LR_CV_L1_result_1)*100) + '%')
print ("Logistic Regression L2: " + str(accuracy(y, LR_CV_result_1)*100) + '%')
print ("Random forest: " + str(accuracy(y, RF_CV_result_1)*100) + '%')
print ("K-nearest-neighbors: " + str(accuracy(y, KNN_CV_result_1)*100) + '%')

Accuracy
Logistic Regression L1: 83.6903547546%
Logistic Regression L2: 83.6903547546%
Random forest: 83.6903547546%
K-nearest-neighbors: 83.2369942197%


In [15]:
forest = RandomForestClassifier()
forest.fit(X1, y)

importances_1 = forest.feature_importances_

print("Feature importance ranking by Random Forest Model:")
for k,v in sorted(zip(map(lambda x: round(x, 4), importances_1), df_user.columns), reverse=True):
    print(v + ": " + str(k))

Feature importance ranking by Random Forest Model:
enabled_for_marketing_drip: 0.1909
GUEST_INVITE: 0.1718
last_session_creation_time: 0.1562
invited_by_user_id: 0.1347
opted_in_to_mailing_list: 0.1337
PERSONAL_PROJECTS: 0.0908
SIGNUP: 0.0622
ORG_INVITE: 0.0596


#### So I removed the 'last login' feature and trained the model again. This time, the accuracy is lower than before. (No surprise, cause I removed the principle feature.) But this time, the importance list gave a more reasonable ranking. And we could notice that for the feature of 'creation_source', the most essential factor is Guest Invite. This gave us a more intuitive way to see the factor could predict future user adoption.